In [1]:
import tweepy
import pandas as pd
import numpy as np
import json
import sys
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from nltk.tokenize import TweetTokenizer 
import nltk

In [2]:
analyzer = SentimentIntensityAnalyzer()
tw = TweetTokenizer()

In [3]:
# Authenticate to Twitter
auth = tweepy.OAuthHandler("", "")
auth.set_access_token("", "")
# Create API object
api = tweepy.API(auth)

In [4]:
#Check Credentials
try:
   api.verify_credentials()
   print("Authentication OK")
except:
   print("Error during authentication")

Authentication OK


## World Series Tweets

In [5]:
twitter_call = api.search(q="WorldSeries", lang="en", rpp=1000, count=500)

In [6]:
tweets = []
for tweet in twitter_call:
    row = {}
    clean = nltk.tokenize.casual.remove_handles(tweet.text)
    row['Tweet'] = clean
    row ['Time'] = tweet.created_at
    row['User Name'] = tweet.user.name
    row['Score'] = analyzer.polarity_scores(clean)
    row['Location'] = tweet.user.location
    tweets.append(row)
    
print(tweets)

[{'Tweet': 'Who is playing in Game 7? NOT the Dodgers, of course. #WorldSeries', 'Time': datetime.datetime(2019, 10, 30, 4, 17, 36), 'User Name': 'Christian Arana', 'Score': {'neg': 0.0, 'neu': 0.859, 'pos': 0.141, 'compound': 0.2023}, 'Location': 'San Francisco, CA'}, {'Tweet': 'RT  : Justin Verlander in the regular season vs. Justin Verlander in the #WorldSeries\xa0: 0-6 38 IP. 5.68 ERA https://t.co/WGklM…', 'Time': datetime.datetime(2019, 10, 30, 4, 17, 36), 'User Name': 'IG: AllEyezOnJesus', 'Score': {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}, 'Location': '3-6 Mafia Boulevard'}, {'Tweet': 'RT  : "I saw that, what Bregman did in the first inning... I was like, \'that was pretty cool, I wanna do that.\'" - Juan Soto\n\n …', 'Time': datetime.datetime(2019, 10, 30, 4, 17, 36), 'User Name': 'Jay Jenkins', 'Score': {'neg': 0.0, 'neu': 0.704, 'pos': 0.296, 'compound': 0.7906}, 'Location': 'Charlotte, NC'}, {'Tweet': 'GAME 7\nFUCK THE ASTROS!!\n#WorldSeries\n#STAYINTHEFIGHT', 'T

In [7]:
world_series_df = pd.DataFrame(tweets)

In [8]:
world_series_list = []
for score in world_series_df['Score'].iteritems():
    world_series_scores = (score[1].get('compound', 0)) 
    world_series_list.append(world_series_scores)

In [9]:
world_series_df['Compound Score'] = world_series_list
world_series_table = world_series_df.drop(columns=['User Name'])
world_series_table

,Tweet,Time,Score,Location,Compound Score
0,"Who is playing in Game 7? NOT the Dodgers, of ...",2019-10-30 04:17:36,"{'neg': 0.0, 'neu': 0.859, 'pos': 0.141, 'comp...","San Francisco, CA",0.2023
1,RT : Justin Verlander in the regular season v...,2019-10-30 04:17:36,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",3-6 Mafia Boulevard,0.0000
2,"RT : ""I saw that, what Bregman did in the fir...",2019-10-30 04:17:36,"{'neg': 0.0, 'neu': 0.704, 'pos': 0.296, 'comp...","Charlotte, NC",0.7906
3,GAME 7\nFUCK THE ASTROS!!\n#WorldSeries\n#STAY...,2019-10-30 04:17:36,"{'neg': 0.491, 'neu': 0.509, 'pos': 0.0, 'comp...",TEXAS,-0.7019
4,This explaination is just as bad as the call. ...,2019-10-30 04:17:36,"{'neg': 0.126, 'neu': 0.791, 'pos': 0.083, 'co...","Minneapolis, Minnesota",-0.2960
5,RT : FINAL: Anthony Rendon and Stephen Strasb...,2019-10-30 04:17:36,"{'neg': 0.0, 'neu': 0.833, 'pos': 0.167, 'comp...","Beautiful Natalia, TX",0.5859
6,RT : History made tonight:\n-Road team won......,2019-10-30 04:17:36,"{'neg': 0.0, 'neu': 0.837, 'pos': 0.163, 'comp...","Arlington, VA",0.5719
7,RT : I will personally fly out and massage Ma...,2019-10-30 04:17:35,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...","In Your Head, Rent Free",0.0000
8,RT : For the 3rd time in #WorldSeries history...,2019-10-30 04:17:35,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",,0.0000
9,RT : The Washington Nationals have been the b...,2019-10-30 04:17:35,"{'neg': 0.0, 'neu': 0.792, 'pos': 0.208, 'comp...","Rockville, MD",0.6369


In [10]:
world_series_table.to_csv('Data/world_series_df.csv')

In [11]:
# Scores categories
positive_WorldSeries = []
negative_WorldSeries = []
neutral_WorldSeries = []

In [12]:
for c_score in world_series_table['Compound Score']:
    if c_score > 0:
        positive_WorldSeries.append(c_score)
    elif c_score < 0:
        negative_WorldSeries.append(c_score)
    else: neutral_WorldSeries.append(c_score)

In [13]:
avg_positive_WorldSeries = sum(positive_WorldSeries) / float(len(positive_WorldSeries))
avg_negative_WorldSeries = sum(negative_WorldSeries) / float(len(negative_WorldSeries))
avg_neutral_WorldSeries = float(len(neutral_WorldSeries))

print(f'The average positive tweets score for World Series is: {avg_positive_WorldSeries}')
print(f'The average negative tweets score for World Series is: {avg_negative_WorldSeries}')
print(f'The total number of neutral tweets for World Series is: {avg_neutral_WorldSeries}')
  

The average positive tweets score for World Series is: 0.5136652173913043
The average negative tweets score for World Series is: -0.3877916666666667
The total number of neutral tweets for World Series is: 30.0


## Astros Tweets

In [14]:
twitter_call = api.search(q="Astros", lang="en", rpp=1000, count=500)

In [15]:
tweets = []
for tweet in twitter_call:
    row = {}
    clean = nltk.tokenize.casual.remove_handles(tweet.text)
    row['Tweet'] = clean
    row ['Time'] = tweet.created_at
    row['User Name'] = tweet.user.name
    row['Score'] = analyzer.polarity_scores(clean)
    row['Location'] = tweet.user.location
    tweets.append(row)
    
print(tweets)

[{'Tweet': 'y’all don’t really like the astros', 'Time': datetime.datetime(2019, 10, 30, 4, 17, 37), 'User Name': 'eduardo 🦇', 'Score': {'neg': 0.0, 'neu': 0.642, 'pos': 0.358, 'compound': 0.4201}, 'Location': 'Texas'}, {'Tweet': 'Houston rockets play the Washington Wizards tomorrow   meanwhile Houston Astros play the Washington Natio… https://t.co/FKtEgUBpZo', 'Time': datetime.datetime(2019, 10, 30, 4, 17, 37), 'User Name': 'Andree🚀', 'Score': {'neg': 0.0, 'neu': 0.73, 'pos': 0.27, 'compound': 0.5859}, 'Location': ''}, {'Tweet': 'GAME 7\nFUCK THE ASTROS!!\n#WorldSeries\n#STAYINTHEFIGHT', 'Time': datetime.datetime(2019, 10, 30, 4, 17, 36), 'User Name': 'The End Of All Things Will Be Televised.', 'Score': {'neg': 0.491, 'neu': 0.509, 'pos': 0.0, 'compound': -0.7019}, 'Location': 'TEXAS '}, {'Tweet': 'RT  : My prediction, game 7:\nNationals 7, Astros 3.', 'Time': datetime.datetime(2019, 10, 30, 4, 17, 36), 'User Name': 'Anthony AO Oropeza', 'Score': {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, '

In [16]:
astros_df = pd.DataFrame(tweets)

In [17]:
# Get compound scores from dictionary
astros_list = []
for score in astros_df['Score'].iteritems():
    astros_scores = (score[1].get('compound', 0)) 
    astros_list.append(astros_scores)

In [18]:
# Create new column with compound scores
astros_df['Compound Score'] = astros_list
astros_table = astros_df.drop(columns=['User Name'])
astros_table

,Tweet,Time,Score,Location,Compound Score
0,y’all don’t really like the astros,2019-10-30 04:17:37,"{'neg': 0.0, 'neu': 0.642, 'pos': 0.358, 'comp...",Texas,0.4201
1,Houston rockets play the Washington Wizards to...,2019-10-30 04:17:37,"{'neg': 0.0, 'neu': 0.73, 'pos': 0.27, 'compou...",,0.5859
2,GAME 7\nFUCK THE ASTROS!!\n#WorldSeries\n#STAY...,2019-10-30 04:17:36,"{'neg': 0.491, 'neu': 0.509, 'pos': 0.0, 'comp...",TEXAS,-0.7019
3,"RT : My prediction, game 7:\nNationals 7, Ast...",2019-10-30 04:17:36,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...","AOART5 - Kansas City, USA",0.0000
4,"Strasburg dominates, Nationals force Game 7. M...",2019-10-30 04:17:36,"{'neg': 0.0, 'neu': 0.87, 'pos': 0.13, 'compou...",Canada,0.0516
5,RT : FINAL: Anthony Rendon and Stephen Strasb...,2019-10-30 04:17:36,"{'neg': 0.0, 'neu': 0.833, 'pos': 0.167, 'comp...","Beautiful Natalia, TX",0.5859
6,RT : rt for astros to win the World Series. H...,2019-10-30 04:17:36,"{'neg': 0.0, 'neu': 0.657, 'pos': 0.343, 'comp...","Houston,Texas",0.7783
7,RT : Nationals beat Astros to force World Ser...,2019-10-30 04:17:36,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",Somewhere on Earth,0.0000
8,RT : For the 3rd time in #WorldSeries history...,2019-10-30 04:17:35,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",,0.0000
9,RT : The day of Game 7 in 2017 was probably m...,2019-10-30 04:17:35,"{'neg': 0.086, 'neu': 0.824, 'pos': 0.09, 'com...","Houston, TX",0.0267


In [19]:
astros_table.to_csv('Data/astros_df.csv')

In [20]:
# Scores categories
positive_astros = []
negative_astros = []
neutral_astros = []

In [21]:
for c_score in astros_table['Compound Score']:
    if c_score > 0:
        positive_astros.append(c_score)
    elif c_score < 0:
        negative_astros.append(c_score)
    else: neutral_astros.append(c_score)


In [22]:
avg_positive_astros = sum(positive_astros) / float(len(positive_astros))
avg_negative_astros = sum(negative_astros) / float(len(negative_astros))
avg_neutral_astros = float(len(neutral_astros))

print(f'The average positive tweets score for Astros is: {avg_positive_astros}')
print(f'The average negative tweets score for Astros is: {avg_negative_astros}')
print(f'The total number of neutral tweets for Astros is: {avg_neutral_astros}')
  

The average positive tweets score for Astros is: 0.4692456521739129
The average negative tweets score for Astros is: -0.33901363636363646
The total number of neutral tweets for Astros is: 32.0


## Nationals Tweets

In [23]:
twitter_call = api.search(q="Nationals", lang="en", rpp=1000, count=500)

In [24]:
tweets = []
for tweet in twitter_call:
    row = {}
    clean = nltk.tokenize.casual.remove_handles(tweet.text)
    row['Tweet'] = clean
    row ['Time'] = tweet.created_at
    row['User Name'] = tweet.user.name
    row['Score'] = analyzer.polarity_scores(clean)
    row['Location'] = tweet.user.location
    tweets.append(row)
    
print(tweets)

[{'Tweet': 'Fired up after a disputed call at first base went against them in the seventh, the Nationals padded their lead over… https://t.co/B3f8R6wqBb', 'Time': datetime.datetime(2019, 10, 30, 4, 17, 38), 'User Name': 'Star Tribune Sports', 'Score': {'neg': 0.24, 'neu': 0.76, 'pos': 0.0, 'compound': -0.7184}, 'Location': 'Minneapolis, MN'}, {'Tweet': 'RT  : Stephen Strasburg delivered eight strong innings in Game 6 of the World Series with the Nationals on the ropes.\n\nhttps://t.c…', 'Time': datetime.datetime(2019, 10, 30, 4, 17, 38), 'User Name': 'Jason White', 'Score': {'neg': 0.0, 'neu': 0.852, 'pos': 0.148, 'compound': 0.5106}, 'Location': 'Anchorage, AK'}, {'Tweet': 'RT  : So proud of our own Santee’s (San Diego) Stephen Strasburg and his 8 1/3 dominate innings pitching the   into…', 'Time': datetime.datetime(2019, 10, 30, 4, 17, 38), 'User Name': 'Phil Stevenson', 'Score': {'neg': 0.069, 'neu': 0.777, 'pos': 0.155, 'compound': 0.4391}, 'Location': 'Roseville, CA'}, {'Tweet': '

In [25]:
nationals_df = pd.DataFrame(tweets)

In [26]:
nationals_list = []
for score in nationals_df['Score'].iteritems():
    nationals_scores = (score[1].get('compound', 0)) 
    nationals_list.append(nationals_scores)

In [27]:
nationals_df['Compound Score'] = nationals_list
nationals_table = nationals_df.drop(columns=['User Name'])
nationals_table

,Tweet,Time,Score,Location,Compound Score
0,Fired up after a disputed call at first base w...,2019-10-30 04:17:38,"{'neg': 0.24, 'neu': 0.76, 'pos': 0.0, 'compou...","Minneapolis, MN",-0.7184
1,RT : Stephen Strasburg delivered eight strong...,2019-10-30 04:17:38,"{'neg': 0.0, 'neu': 0.852, 'pos': 0.148, 'comp...","Anchorage, AK",0.5106
2,RT : So proud of our own Santee’s (San Diego)...,2019-10-30 04:17:38,"{'neg': 0.069, 'neu': 0.777, 'pos': 0.155, 'co...","Roseville, CA",0.4391
3,RT : The Washington Nationals have been the b...,2019-10-30 04:17:38,"{'neg': 0.0, 'neu': 0.792, 'pos': 0.208, 'comp...","Charlottesville, VA",0.6369
4,RT : Went 1-0 today.\n\n#WorldSeries // #STAY...,2019-10-30 04:17:38,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...","Florida, USA",0.0000
5,RT : HAMMER the Nationals tomorrow https://...,2019-10-30 04:17:36,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",dreamworld .1,0.0000
6,"RT : ""I saw that, what Bregman did in the fir...",2019-10-30 04:17:36,"{'neg': 0.0, 'neu': 0.704, 'pos': 0.296, 'comp...","Charlotte, NC",0.7906
7,"RT : My prediction, game 7:\nNationals 7, Ast...",2019-10-30 04:17:36,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...","AOART5 - Kansas City, USA",0.0000
8,"Strasburg dominates, Nationals force Game 7. M...",2019-10-30 04:17:36,"{'neg': 0.0, 'neu': 0.87, 'pos': 0.13, 'compou...",Canada,0.0516
9,RT : FINAL: Anthony Rendon and Stephen Strasb...,2019-10-30 04:17:36,"{'neg': 0.0, 'neu': 0.833, 'pos': 0.167, 'comp...","Beautiful Natalia, TX",0.5859


In [28]:
nationals_table.to_csv('Data/nationals_df.csv')

In [29]:
# Scores categories
positive_nationals = []
negative_nationals = []
neutral_nationals = []

In [30]:
for c_score in nationals_table['Compound Score']:
    if c_score > 0:
        positive_nationals.append(c_score)
    elif c_score < 0:
        negative_nationals.append(c_score)
    else: neutral_nationals.append(c_score)

In [31]:
avg_positive_nationals = sum(positive_nationals) / float(len(positive_nationals))
avg_negative_nationals = sum(negative_nationals) / float(len(negative_nationals))
avg_neutral_nationals = float(len(neutral_nationals))

print(f'The average positive tweets score for Nationals is: {avg_positive_nationals}')
print(f'The average negative tweets score for Nationals is: {avg_negative_nationals}')
print(f'The total number of neutral tweets for Nationals is: {avg_neutral_nationals}')
  

The average positive tweets score for Nationals is: 0.5443956521739132
The average negative tweets score for Nationals is: -0.33439655172413796
The total number of neutral tweets for Nationals is: 25.0
